In [68]:
import spacy
import pandas as pd
import numpy as np

In [2]:
from collections import Counter
from string import punctuation

In [3]:
nlp = spacy.load("en_core_web_lg")

In [41]:
music_words= ['album', 'genre', 'music', 'albums', 'song', 'songs', 'genres', 'year', 'minutes', 'minute', 'years']
def get_hotwords(text, x, title, artist):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] # 1
    doc = nlp(text.lower()) # 2
    for token in doc:
        # 3
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        # 4
        if(token.text == title or token.text in artist.replace(',', '').split()):
            continue
            
        if(token.text in music_words):
            continue
            
        if(token.pos_ in pos_tag):
            result.append(token.text)
            
    out = Counter(result).most_common(x)
                
    return out

In [6]:
output = get_hotwords('''Welcome to Medium! Medium is a publishing platform where people can read important, insightful stories on the topics that matter most to them and share ideas with the world.''', 5)

In [7]:
print(output)

[('medium', 2), ('welcome', 1), ('publishing', 1), ('platform', 1), ('people', 1)]


In [49]:
data = pd.read_json("../solrData/clean_reviews.json", orient='records')
data

,reviewid,title,artist,score,author,review_publication_date,genre,song_release_year,record_label,review_content
0,22703,mezzanine,massive attack,9.3,nate patrin,1483833600000,electronic,1998,virgin,"“Trip-hop” eventually became a ’90s punchline,..."
1,22721,prelapsarian,krallice,7.9,zoe camp,1483747200000,metal,2016,hathenter,"Eight years, five albums, and two EPs in, the ..."
2,22659,all of them naturals,uranium club,7.3,david glickman,1483747200000,rock,2016,"static shock,fashionable idiots",Minneapolis’ Uranium Club seem to revel in bei...
3,22661,first songs,"kleenex, liliput",9.0,jenn pelly,1483660800000,rock,2016,"kill rock stars,mississippi",Kleenex began with a crash. It transpired one ...
4,22725,new start,taso,8.1,kevin lozano,1483660800000,electronic,2016,teklife,It is impossible to consider a given release b...
...,...,...,...,...,...,...,...,...,...,...
18412,1535,let us replay!,coldcut,8.9,james p. wisdom,917308800000,"jazz,electronic",0,ninja tune,The marketing guys of yer average modern megac...
18413,1341,1999,cassius,4.8,james p. wisdom,917308800000,electronic,0,astralwerks,"Well, it's been two weeks now, and I guess it'..."
18414,5376,out of tune,mojave 3,6.3,jason josephes,916099200000,rock,0,4ad,"Out of Tune is a Steve Martin album. Yes, I'l..."
18415,2413,"singles breaking up, vol. 1",don caballero,7.2,james p. wisdom,916099200000,"rock,metal,experimental",1999,touch and go,"Well, kids, I just went back and re-read my re..."


In [45]:
content = list(data['review_content'])
titles = list(data['title'])
artist = list(data['artist'])
key_words = []
for i in range(len(content)):
    result = get_hotwords(content[i], 25, titles[i], artist[i])
    key_words.append(result)
    

In [51]:
data['key_words'] = key_words[:, 0]
data

TypeError: list indices must be integers or slices, not tuple

In [47]:
data.to_json('reviews_with_keywordsNr.json', orient='records')

In [80]:
print(len(key_words))

18417


In [81]:
only_words = []
only_numbers = []

for i in range(len(key_words)):
    a = key_words[i]
    if not a:
        only_numbers.append([])
        only_words.append([])
        continue
    number = list(list(zip(*a))[1])
    words = list(list(zip(*a))[0])
    only_numbers.append(number)
    only_words.append(words)
    

    

In [82]:
data['keywords'] = only_words
data['keyword_frequency'] = only_numbers

In [83]:
data

,reviewid,title,artist,score,author,review_publication_date,genre,song_release_year,record_label,review_content,keywords,keyword_frequency
0,22703,mezzanine,massive attack,9.3,nate patrin,1483833600000,electronic,1998,virgin,"“Trip-hop” eventually became a ’90s punchline,...","[late, del, naja, hop, band, 3d, andy, man, tr...","[5, 5, 5, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, ..."
1,22721,prelapsarian,krallice,7.9,zoe camp,1483747200000,metal,2016,hathenter,"Eight years, five albums, and two EPs in, the ...","[band, barr, metal, post, mcmaster, winter, de...","[4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
2,22659,all of them naturals,uranium club,7.3,david glickman,1483747200000,rock,2016,"static shock,fashionable idiots",Minneapolis’ Uranium Club seem to revel in bei...,"[punk, band, human, exploration, ep, scene, na...","[7, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,22661,first songs,"kleenex, liliput",9.0,jenn pelly,1483660800000,rock,2016,"kill rock stars,mississippi",Kleenex began with a crash. It transpired one ...,"[punk, marder, scene, beri, early, chords, dru...","[9, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, ..."
4,22725,new start,taso,8.1,kevin lozano,1483660800000,electronic,2016,teklife,It is impossible to consider a given release b...,"[footwork, double, cup, rashad, way, start, tr...","[11, 7, 6, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
18412,1535,let us replay!,coldcut,8.9,james p. wisdom,917308800000,"jazz,electronic",0,ninja tune,The marketing guys of yer average modern megac...,"[brand, equity, pale, new, money, product, goo...","[7, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
18413,1341,1999,cassius,4.8,james p. wisdom,917308800000,electronic,0,astralwerks,"Well, it's been two weeks now, and I guess it'...","[techno, sure, club, weeks, time, guys, dilige...","[3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
18414,5376,out of tune,mojave 3,6.3,jason josephes,916099200000,rock,0,4ad,"Out of Tune is a Steve Martin album. Yes, I'l...","[martin, man, slowdive, steve, funny, laughter...","[8, 5, 5, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, ..."
18415,2413,"singles breaking up, vol. 1",don caballero,7.2,james p. wisdom,916099200000,"rock,metal,experimental",1999,touch and go,"Well, kids, I just went back and re-read my re...","[sore-, jamming, review, sharp, drums, oppress...","[4, 4, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, ..."


In [84]:
data.to_json('reviews_with_keywordsNr.json', orient='records')

In [87]:
data = data.drop(columns=['keyword_frequency'])
data.to_json('reviews_with_only_keywords25.json', orient='records')

In [88]:
data

,reviewid,title,artist,score,author,review_publication_date,genre,song_release_year,record_label,review_content,keywords
0,22703,mezzanine,massive attack,9.3,nate patrin,1483833600000,electronic,1998,virgin,"“Trip-hop” eventually became a ’90s punchline,...","[late, del, naja, hop, band, 3d, andy, man, tr..."
1,22721,prelapsarian,krallice,7.9,zoe camp,1483747200000,metal,2016,hathenter,"Eight years, five albums, and two EPs in, the ...","[band, barr, metal, post, mcmaster, winter, de..."
2,22659,all of them naturals,uranium club,7.3,david glickman,1483747200000,rock,2016,"static shock,fashionable idiots",Minneapolis’ Uranium Club seem to revel in bei...,"[punk, band, human, exploration, ep, scene, na..."
3,22661,first songs,"kleenex, liliput",9.0,jenn pelly,1483660800000,rock,2016,"kill rock stars,mississippi",Kleenex began with a crash. It transpired one ...,"[punk, marder, scene, beri, early, chords, dru..."
4,22725,new start,taso,8.1,kevin lozano,1483660800000,electronic,2016,teklife,It is impossible to consider a given release b...,"[footwork, double, cup, rashad, way, start, tr..."
...,...,...,...,...,...,...,...,...,...,...,...
18412,1535,let us replay!,coldcut,8.9,james p. wisdom,917308800000,"jazz,electronic",0,ninja tune,The marketing guys of yer average modern megac...,"[brand, equity, pale, new, money, product, goo..."
18413,1341,1999,cassius,4.8,james p. wisdom,917308800000,electronic,0,astralwerks,"Well, it's been two weeks now, and I guess it'...","[techno, sure, club, weeks, time, guys, dilige..."
18414,5376,out of tune,mojave 3,6.3,jason josephes,916099200000,rock,0,4ad,"Out of Tune is a Steve Martin album. Yes, I'l...","[martin, man, slowdive, steve, funny, laughter..."
18415,2413,"singles breaking up, vol. 1",don caballero,7.2,james p. wisdom,916099200000,"rock,metal,experimental",1999,touch and go,"Well, kids, I just went back and re-read my re...","[sore-, jamming, review, sharp, drums, oppress..."
